In [42]:
import pandas as pd
import numpy as np
import csv

import seaborn as sns
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim


from glob import glob
from tqdm.cli import tqdm
import re, os, random


from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as v2
import timm

from PIL import Image

from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [43]:
def seed_everything(seed=42):
    random.seed(seed)  # Python 내장 random 모듈
    os.environ['PYTHONHASHSEED'] = str(seed)  # 환경변수 설정
    np.random.seed(seed)  # NumPy
    torch.manual_seed(seed)  # PyTorch CPU 시드 고정
    torch.cuda.manual_seed(seed)  # PyTorch GPU 시드 고정
    torch.cuda.manual_seed_all(seed)  # 멀티 GPU 환경에서도 시드 고정
    torch.backends.cudnn.deterministic = True  # CuDNN 관련 설정
    torch.backends.cudnn.benchmark = False  # 동일한 입력 크기의 데이터가 반복될 경우 속도 향상을 위한 벤치마크 모드 비활성화

# 사용 예시
seed_everything(seed=42)

이미지 경로 포함

### 데이터 mean, std 계산

In [44]:
import numpy as np
import albumentations as A
from albumentations.core.transforms_interface import ImageOnlyTransform
from albumentations.pytorch import ToTensorV2

# train기준 mean/std # leakage 아님.
mean = 15.359733188267395
std = 58.28444875927197

test_transforms = A.Compose([
    A.Resize(always_apply = True, p=1.0, height=224, width=224),
    A.Normalize(mean=[mean, mean, mean], std=[std, std, std], p=1.0),
    ToTensorV2()
])

In [45]:
test_df = pd.read_csv('../../../origin_datasets/answer_sample.csv')
# test_df = pd.read_csv('/home/hdd1/ICTC2024/answer_sample.csv')
test_df['data_path'] = test_df.apply(lambda row: f"../new_image_dataset/user{row['subject_id']}_{row['date']}_test_all_sensor.png", axis=1)
# test_df['data_path'] = test_df.apply(lambda row: f"/home/hdd1/ICTC2024/huni_code/code/image_si/new_image_dataset/user{row['subject_id']}_{row['date']}_test.png", axis=1)

In [46]:
class CustomDataset(Dataset):
    def __init__(self, df, transforms):
        self.path = df['data_path'].values
        self.transform = transforms

    def __getitem__(self, idx):
        try:
            img = np.array(Image.open(self.path[idx]).convert('RGB'))
        except FileNotFoundError:
            # If file not found, skip to the next item
            return self.__getitem__((idx + 1) % len(self))
        
        img = self.transform(image=img)
        img = img["image"]
        
        return img

    def __len__(self):
        return len(self.path)


In [47]:
test_dataset = CustomDataset(test_df, test_transforms)

In [48]:
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

In [49]:
# model = timm.create_model('deit3_large_patch16_384', pretrained=False, num_classes=7)
# checkpoint = torch.load(f'models/model_fold_0_best.pt')
# model.load_state_dict(checkpoint)
# model = model.to('cuda')
# model.eval()

In [50]:
# def predict(model, test_loader, device):
#     model.eval()
#     model_pred = []
#     with torch.no_grad():
#         for data in tqdm(iter(test_loader)):
#             data = data.to(device)
#             pred = model(data)  # Get raw model predictions
#             pred = torch.sigmoid(pred)
#             model_pred.extend(pred.cpu().tolist())  # Append predictions to the list

#     return model_pred

In [56]:
import timm
import torch
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

models = []
for fold in ['model_fold_0_f1-0.686_loss-0.839_reverseQ3', 'model_fold_1_f1-0.756_loss-0.751_reverseQ3', 'model_fold_2_f1-0.788_loss-0.651_reverseQ3', 'model_fold_3_f1-0.741_loss-0.795_reverseQ3', 'model_fold_4_f1-0.767_loss-0.667_reverseQ3']:
    model = timm.create_model('resnext101_32x32d', pretrained=False, num_classes=7)
    checkpoint = torch.load(f'{fold}.pt')
    model.load_state_dict(checkpoint)
    model = model.to('cuda')
    model.eval()
    models.append(model)

def predict_ensemble_hard_voting(models, test_loader, device, threshold=0.5):
    model_preds = []
    with torch.no_grad():
        for data in tqdm(iter(test_loader)):
            data = data.to(device)
            fold_preds = []
            for model in models:
                pred = model(data)  # Get raw model predictions
                pred = torch.sigmoid(pred)
                pred = (pred > threshold).cpu().numpy().astype(int)  # Binarize predictions
                fold_preds.append(pred)
            # Hard voting by majority rule
            fold_preds = np.array(fold_preds)
            final_pred = np.apply_along_axis(lambda x: np.bincount(x, minlength=2).argmax(), axis=0, arr=fold_preds)  # Majority vote
            model_preds.extend(final_pred)
    return model_preds

preds = np.array(predict_ensemble_hard_voting(models, test_loader, 'cuda'))
predictions_df = pd.DataFrame(preds, columns=['Q1', 'Q2', 'Q3', 'S1', 'S2', 'S3', 'S4'])

# Assuming `test_df` is your original DataFrame with test data
final_df = pd.concat([test_df[['subject_id', 'date']], predictions_df], axis=1)

# Optionally save or display the DataFrame
final_df.to_csv(f"binary_predictions_hard_voting.csv", index=False)


  0%|          | 0/15 [00:00<?, ?it/s]

In [57]:
read = pd.read_csv('binary_predictions1.csv')
read.head()

,subject_id,date,Q1,Q2,Q3,S1,S2,S3,S4
0,5,2023-11-05,0,1,0,1,0,0,0
1,5,2023-11-06,0,1,0,0,0,0,0
2,5,2023-11-07,1,0,0,1,1,1,0
3,5,2023-11-08,0,0,0,0,0,0,0
4,5,2023-11-09,0,1,0,1,0,0,0


In [58]:
print(final_df.head(60))

    subject_id        date  Q1  Q2  Q3  S1  S2  S3  S4
0            5  2023-11-05   1   1   1   1   0   0   0
1            5  2023-11-06   0   0   1   0   0   0   0
2            5  2023-11-07   1   1   1   1   1   1   0
3            5  2023-11-08   0   0   1   0   0   0   0
4            5  2023-11-09   0   1   1   0   0   0   0
5            5  2023-11-10   0   1   1   0   0   0   0
6            5  2023-11-11   0   0   1   0   0   0   0
7            5  2023-11-12   0   0   1   0   0   0   0
8            5  2023-11-13   1   1   1   1   0   0   0
9            5  2023-11-14   0   0   1   0   0   0   0
10           5  2023-11-15   0   0   1   0   0   0   0
11           5  2023-11-16   0   0   1   0   0   0   0
12           5  2023-11-17   0   0   1   0   0   0   0
13           5  2023-11-18   0   0   1   0   0   0   0
14           5  2023-11-19   0   1   1   0   0   0   0
15           5  2023-11-20   0   0   1   0   0   0   0
16           5  2023-11-21   0   1   1   0   0   0   0
17        

In [59]:
print(final_df['Q3'].value_counts())

1    91
0    24
Name: Q3, dtype: int64


In [26]:
final_df['Q3'] = final_df['Q3'].apply(lambda x: 1 if x == 0 else 0)

In [28]:
final_df.head(50)

,subject_id,date,Q1,Q2,Q3,S1,S2,S3,S4
0,5,2023-11-05,0,1,1,0,0,0,0
1,5,2023-11-06,0,1,1,0,0,0,0
2,5,2023-11-07,0,0,1,0,1,0,1
3,5,2023-11-08,0,1,1,0,0,0,0
4,5,2023-11-09,0,1,1,0,0,0,0
5,5,2023-11-10,1,0,1,0,0,0,0
6,5,2023-11-11,1,1,1,0,0,0,0
7,5,2023-11-12,0,0,1,0,0,1,0
8,5,2023-11-13,0,0,1,0,0,0,0
9,5,2023-11-14,0,1,1,0,0,0,0


In [29]:

final_df.to_csv(f"ensemble_resnext101_Q3reverse.csv", index=False)

# XAI

In [ ]:
!pip install grad-cam

In [ ]:
model = timm.create_model('vit_huge_patch14_224', pretrained=False, num_classes=7)
model = model.to('cuda')
model.load_state_dict(torch.load('./models/base_image.pt'))
model.eval()

In [ ]:
# last_conv_layer = model.stages[3].blocks[-1].norm

In [ ]:
from pytorch_grad_cam import GradCAM, HiResCAM, ScoreCAM, GradCAMPlusPlus, AblationCAM, XGradCAM, EigenCAM, FullGrad
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image
from torchvision.models import resnet50

target_layers = [model.layers[-1].blocks[-1].norm2]
import torch
from PIL import Image
from torchvision import transforms

# 이미지 파일 경로
image_path = '..//new_image_dataset/user8_2023-10-23_test.png'

# 이미지를 불러오기
img = np.array(Image.open(image_path).convert('RGB'))
img = img / 255
# 모델에 입력하기 위한 전처리 과정 정의
preprocess = transforms.Compose([
    transforms.Resize(384),
    transforms.ToTensor(),
#     transforms.Normalize(mean=[mean, mean, mean], std=[std, std, std]),
    
])

# 이미지를 전처리하고 Tensor로 변환
input_tensor = preprocess(image)

# 배치 차원 추가 (모델은 일반적으로 배치를 입력으로 받음)
rgb_img = input_tensor.unsqueeze(0)
# Create an input tensor image for your model..
# Note: input_tensor can be a batch tensor with several images!

# Construct the CAM object once, and then re-use it on many images:
cam = FullGrad(model=model, target_layers=target_layers)

# You can also use it within a with statement, to make sure it is freed,
# In case you need to re-create it inside an outer loop:
# with GradCAM(model=model, target_layers=target_layers) as cam:
#   ...

# We have to specify the target we want to generate
# the Class Activation Maps for.
# If targets is None, the highest scoring category
# will be used for every image in the batch.
# Here we use ClassifierOutputTarget, but you can define your own custom targets
# That are, for example, combinations of categories, or specific outputs in a non standard model.

targets = [ClassifierOutputTarget(1)]

# You can also pass aug_smooth=True and eigen_smooth=True, to apply smoothing.
grayscale_cam = cam(input_tensor=input_tensor, targets=targets)

# In this example grayscale_cam has only one image in the batch:
grayscale_cam = grayscale_cam[0, :]
visualization = show_cam_on_image(img, grayscale_cam, use_rgb=True)

# You can also get the model outputs without having to re-inference
model_outputs = cam.outputs

In [ ]:
valid_loss, test_acc, test_f1 = run_model(model, test_loader, criterion, optimizer, is_training=False)
test_acc, test_f1